In [1]:
import numpy as np
import warnings
warnings.filterwarnings("ignore")
X = {}
y = {}
X["train"] = np.load("X_train.npy")
y["train"] = np.load("y_train.npy")
X["dev"] = np.load("X_dev.npy")
y["dev"] = np.load("y_dev.npy")
X_text_train = np.load("X_train_text.npy")
X_text_dev = np.load("X_test_text.npy")
X["train"].shape, X["dev"].shape

((9632, 300), (1071, 300))

In [2]:
from sklearn.linear_model import LogisticRegression
l2 = 10
model = LogisticRegression(penalty='l2', C=1/l2)
model.fit(X["train"], y["train"])
model.score(X["dev"], y["dev"])

0.8860877684407096

In [3]:
w = np.concatenate((model.coef_, model.intercept_[None, :]), axis=1)
F_train = np.concatenate([X["train"], np.ones((X["train"].shape[0], 1))], axis=1) # Concatenating one to calculate the gradient with respect to intercept
F_dev = np.concatenate([X["dev"], np.ones((X["dev"].shape[0], 1))], axis=1)

error_train = model.predict_proba(X["train"])[:, 1] - y["train"]
error_dev = model.predict_proba(X["dev"])[:, 1] - y["dev"]

gradient_train = F_train * error_train[:, None]
gradient_dev = F_dev * error_dev[:, None]
gradient_train.shape, gradient_dev.shape

((9632, 301), (1071, 301))

In [4]:
#from scipy import sparse
probs = model.predict_proba(X["train"])[:, 1]
hessian = F_train.T @ np.diag(probs * (1 - probs)) @ F_train / X["train"].shape[0] + l2 * np.eye(F_train.shape[1]) / X["train"].shape[0]
inverse_hessian = np.linalg.inv(hessian)

In [5]:
eps = 1 / X["train"].shape[0]
delta_k = -eps * inverse_hessian @ gradient_train.T
pred = np.reshape(model.predict_proba(X["dev"])[:, 1], (model.predict_proba(X["dev"])[:, 1].shape[0], 1))
grad_f = F_dev * (pred * (1 - pred))
delta_pred = grad_f @ delta_k
delta_pred.shape

(1071, 9632)

In [6]:
np.save("old_predictions.npy", pred)

In [7]:
def Remove(k, scores, test_idx):
    #print("test_idx", test_idx)
    #print("old")
    #print(pred[test_idx])
    
    if pred[test_idx] > 0.5:
        top_k_index = scores[test_idx].argsort()[-k:]
    else:
        top_k_index = scores[test_idx].argsort()[:k]

    X_k = np.delete(X["train"], top_k_index, axis=0)
    y_k = np.delete(y["train"], top_k_index, axis=0)

        
    prediction = -np.sum(scores[test_idx][top_k_index])
    print("prediction", prediction)

    return X_k, y_k, prediction

In [8]:
ls

CSV_emo_LR_alg1.ipynb   X_dev.npy               new_pred.npy
CSV_speech_LR_alg1.csv  X_test_text.npy         npy2tsv.ipynb
Data_loader.ipynb       X_train.npy             old_predictions.npy
LR_loan_normal.ipynb    X_train_text.npy        y_dev.npy
Smallest_k2.ipynb       appro_ks.npy            y_test_text.npy
Smallest_k2.py          essay_data/             y_train.npy
Use_approK.ipynb        hate_speech18/          y_train_text.npy


In [9]:
app_k = np.array(np.load("appro_ks.npy", allow_pickle=True).squeeze())
app_k_nonon=[]
for i in range(X["dev"].shape[0]):
    print(app_k[i])
    if (not app_k[i] > 0):
        app_k_nonon.append(X["train"].shape[0])
    else:
        app_k_nonon.append(app_k[i])

2191.0
24.0
nan
nan
72.0
19.0
3.0
518.0
237.0
nan
904.0
907.0
118.0
269.0
665.0
70.0
31.0
145.0
4568.0
nan
nan
nan
51.0
79.0
155.0
4314.0
1830.0
nan
1326.0
269.0
901.0
nan
2845.0
nan
nan
192.0
nan
336.0
nan
847.0
nan
nan
232.0
4543.0
49.0
426.0
nan
15.0
49.0
2.0
1130.0
809.0
1274.0
nan
117.0
nan
nan
13.0
285.0
884.0
513.0
nan
1985.0
nan
nan
1051.0
nan
240.0
4784.0
797.0
1011.0
nan
1846.0
1912.0
nan
73.0
154.0
49.0
783.0
nan
nan
nan
31.0
509.0
431.0
nan
51.0
35.0
15.0
234.0
nan
nan
nan
nan
1512.0
258.0
nan
nan
nan
nan
1775.0
352.0
nan
nan
48.0
nan
971.0
105.0
289.0
nan
3078.0
865.0
800.0
78.0
1511.0
316.0
259.0
nan
1443.0
583.0
nan
2190.0
nan
24.0
nan
2.0
nan
909.0
17.0
nan
1197.0
178.0
nan
1.0
31.0
433.0
nan
nan
nan
304.0
336.0
nan
472.0
82.0
nan
628.0
nan
432.0
32.0
12.0
368.0
5.0
18.0
1766.0
nan
296.0
8.0
1830.0
103.0
194.0
365.0
1468.0
nan
nan
nan
9.0
164.0
45.0
70.0
45.0
20.0
761.0
nan
nan
nan
11.0
nan
79.0
801.0
nan
40.0
2201.0
168.0
206.0
nan
nan
407.0
1289.0
nan
45.0
184.0
nan
n

In [10]:
sort_dev_k = np.array(app_k_nonon).argsort()
sort_dev_k

array([ 274,  537,  514, ...,  409,  876, 1070])

In [11]:
app_k_nonon[sort_dev_k[0]]

1.0

In [12]:
header = ["test_movie_id", "test_movie_review_text", "old_prediction", "right_prediction", "k", "successful_flip"]
for i in range(X["train"].shape[0]):
    header.append("train_example_" + str(i+1))

In [13]:
new_predictions = np.load("new_pred.npy", allow_pickle=True)

In [ ]:
data = []
for test_movie_id in sort_dev_k:
    
    test_point = []
    # id
    test_point.append(test_movie_id)
    # test_movie_review_text
    test_point.append("label: " + str(y["dev"][test_movie_id]) + "  doc: " + str(X_text_dev[test_movie_id]))
    test_point.append(pred[test_movie_id].item())
    test_point.append((pred[test_movie_id].item() > 0.25) == y["dev"][test_movie_id].item())
    # k
    k = app_k[test_movie_id]
    test_point.append(str(k))
    print("k", k)
    if not (k >= 0):
        test_point.append(None)
        data.append(test_point)
        continue
    k = int(k)
    test_point.append((pred[test_movie_id].item()>0.25) != (new_predictions[test_movie_id].item()>0.25))
        
    
    if pred[test_movie_id] > 0.25:
        top_k_index = delta_pred[test_movie_id].argsort()[-k:]
    else:
        top_k_index = delta_pred[test_movie_id].argsort()[:k]
    
    # training examples
    for idx in top_k_index:
        test_point.append("id: " + str(idx) + "  label: " + str(y["train"][idx]) + "  doc: " + str(X_text_train[idx]))
    data.append(test_point)

k 1.0
k 1.0
k 1.0
k 1.0
k 1.0
k 1.0
k 1.0
k 1.0
k 1.0
k 2.0
k 2.0
k 2.0
k 2.0
k 2.0
k 2.0
k 3.0
k 3.0
k 3.0
k 3.0
k 3.0
k 3.0
k 3.0
k 3.0
k 3.0
k 4.0
k 5.0
k 5.0
k 5.0
k 6.0
k 6.0
k 7.0
k 7.0
k 7.0
k 8.0
k 8.0
k 8.0
k 9.0
k 9.0
k 9.0
k 9.0
k 10.0
k 10.0
k 10.0
k 11.0
k 11.0
k 11.0
k 11.0
k 11.0
k 12.0
k 12.0
k 12.0
k 12.0
k 12.0
k 13.0
k 13.0
k 14.0
k 14.0
k 14.0
k 15.0
k 15.0
k 16.0
k 16.0
k 16.0
k 16.0
k 16.0
k 16.0
k 17.0
k 18.0
k 18.0
k 18.0
k 19.0
k 20.0
k 20.0
k 21.0
k 22.0
k 23.0
k 23.0
k 24.0
k 24.0
k 24.0
k 24.0
k 24.0
k 25.0
k 26.0
k 26.0
k 26.0
k 26.0
k 27.0
k 28.0
k 28.0
k 28.0
k 28.0
k 29.0
k 31.0
k 31.0
k 31.0
k 31.0
k 31.0
k 31.0
k 31.0
k 31.0
k 32.0
k 32.0
k 32.0
k 34.0
k 34.0
k 34.0
k 34.0
k 34.0
k 35.0
k 35.0
k 36.0
k 38.0
k 38.0
k 39.0
k 40.0
k 40.0
k 40.0
k 40.0
k 40.0
k 41.0
k 41.0
k 42.0
k 42.0
k 43.0
k 44.0
k 44.0
k 44.0
k 45.0
k 45.0
k 45.0
k 46.0
k 46.0
k 47.0
k 48.0
k 48.0
k 48.0
k 49.0
k 49.0
k 49.0
k 49.0
k 49.0
k 49.0
k 51.0
k 51.0
k 51.0
k 52.0
k 53.0
k 54

In [ ]:
import csv

with open('CSV_speech_LR_alg1.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(data)


In [ ]:
ls

In [ ]:
pwd